In [0]:
import time
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession 
from pyspark.sql.functions import *
from pyspark.sql.types import ArrayType,IntegerType,MapType

In [0]:
#attention fonctionnement différent de RDD / iic la fonction prend en argument la colonne spécifiée et non (key,value)
#il faut passer Key en paramètre
import time
#@udf(MapType(IntegerType(),IntegerType()))
@udf(ArrayType(ArrayType(IntegerType())))
def DFreducerBis(key,value):
    min=key
    valuesList=[]
    listoutput=[]
#   global accu
    for i in value:
        if i <min: 
            min=i
        valuesList.append(i)
    if min < key:
        listoutput.append((0,key,min))
        for j in valuesList:
            if min != j:
              listoutput.append((1,j,min))
#             accu.add(1)
#   print(listoutput)
    return listoutput

In [0]:
path = "/FileStore/tables/RDD/" 
DF_Init=spark.read.csv(path + "/facebook_combined.txt",sep=" ")


In [0]:
DF_Init.count()

Out[18]: 88234

In [0]:
#essai programme complet n°1
#formatage initial du fichier  / 88 234 edges
DF=DF_Init.withColumn("_c0",DF_Init["_c0"].cast("int")).withColumnRenamed("_c0","key")\
                     .withColumn("_c1",DF_Init["_c1"].cast("int")).withColumnRenamed("_c1","value")

#Initialisation de la boucle
boucle = True
t0=time.time()

while boucle == True:

  #map du CFF
  MapDF=DF.union(DF.select('value','key')).groupBy("key").agg(collect_list("value").alias("MapOutput"))

  #fonction reduce du CCF Iterate DF fichier de chiffres
  ReduceDF=MapDF.withColumn("ListnewPairs",DFreducerBis("key","MapOutput")).select(explode(col('ListnewPairs'))).select([col("col")[i] for i in range(3)])

  compteur=ReduceDF.where("col[0]=1").count()
  print(compteur)

  ##suite #fonction Reduce de CFF Dedup sur base RDD liste de chiffres pour vérifier la suppression des doublons de tuples
  DF=ReduceDF.withColumnRenamed("col[1]","key").withColumnRenamed("col[2]","value").select("key","value").distinct()

  if compteur == 0:
    print("arret")
    boucle = False

t1=time.time()
#le round de python interfère l'opérateur round de pspark sql (qui s'applique à des colonnes)
print("c'est la fin; durée totale de :",t1-t0)

171914
18583
7052
1507
304
0
arret
c'est la fin; durée totale de : 51.70449709892273


In [0]:
path = "/FileStore/tables/RDD/" 
DF_Prep=spark.read.csv(path + "/HR_edges.csv",sep=",")
DF_Init=DF_Prep.where("_c0 != 'node_1'")

In [0]:
DF_Init.show(10)

+---+-----+
|_c0|  _c1|
+---+-----+
|  0| 4076|
|  0|29861|
|  0|53717|
|  0| 2382|
|  0|39945|
|  0|22224|
|  0|17332|
|  0|  226|
|  0|30409|
|  0|11699|
+---+-----+
only showing top 10 rows



In [0]:
#essai programme complet n°2
#fichier  498 203 paires de edges
#formatage initial du fichier 
DF=DF_Init.withColumn("_c0",DF_Init["_c0"].cast("int")).withColumnRenamed("_c0","key")\
                     .withColumn("_c1",DF_Init["_c1"].cast("int")).withColumnRenamed("_c1","value")

#Initialisation de la boucle
boucle = True
t0=time.time()

while boucle == True:

  #map du CFF
  MapDF=DF.union(DF.select('value','key')).groupBy("key").agg(collect_list("value").alias("MapOutput"))

  #fonction reduce du CCF Iterate RDD fichier de chiffres
  ReduceDF=MapDF.withColumn("ListnewPairs",DFreducerBis("key","MapOutput")).select(explode(col('ListnewPairs'))).select([col("col")[i] for i in range(3)])

  compteur=ReduceDF.where("col[0]=1").count()
  print(compteur)

  ##suite #fonction Reduce de CFF Dedup sur base RDD liste de chiffres pour vérifier la suppression des doublons de tuples
  DF=ReduceDF.withColumnRenamed("col[1]","key").withColumnRenamed("col[2]","value").select("key","value").distinct()

  if compteur == 0:
    print("arret")
    boucle = False

t1=time.time()
#le round de python interfère l'opérateur round de pspark sql (qui s'applique à des colonnes)
print("c'est la fin; durée totale de :",t1-t0)

913541
1450180
1515978
116067
216
0
arret
c'est la fin; durée totale de : 144.9048490524292


In [0]:
path = "/FileStore/tables/RDD/" 
DF_Prep=spark.read.csv(path + "/large_twitch_edges.csv",sep=",")
DF_Init=DF_Prep.where("_c0 != 'numeric_id_1'")

In [0]:
DF_Init.show(10)

+-----+------+
|  _c0|   _c1|
+-----+------+
|98343|141493|
|98343| 58736|
|98343|140703|
|98343|151401|
|98343|157118|
|98343|125430|
|98343|  3635|
|98343|   495|
|98343|116648|
|98343|  1679|
+-----+------+
only showing top 10 rows



In [0]:
DF_Init.count()

Out[23]: 6797557

In [0]:
#essai programme complet n°4
#chargement du fichier 6 797 557 paires de edges

#formatage initial du fichier 
DF=DF_Init.withColumn("_c0",DF_Init["_c0"].cast("int")).withColumnRenamed("_c0","key")\
                     .withColumn("_c1",DF_Init["_c1"].cast("int")).withColumnRenamed("_c1","value")

#Initialisation de la boucle
boucle = True
t0=time.time()

while boucle == True:

  #map du CFF
  MapDF=DF.union(DF.select('value','key')).groupBy("key").agg(collect_list("value").alias("MapOutput"))

  #fonction reduce du CCF Iterate RDD fichier de chiffres
  ReduceDF=MapDF.withColumn("ListnewPairs",DFreducerBis("key","MapOutput")).select(explode(col('ListnewPairs'))).select([col("col")[i] for i in range(3)])

  compteur=ReduceDF.where("col[0]=1").count()
  print(compteur)

  ##suite #fonction Reduce de CFF Dedup sur base RDD liste de chiffres pour vérifier la suppression des doublons de tuples
  DF=ReduceDF.withColumnRenamed("col[1]","key").withColumnRenamed("col[2]","value").select("key","value").distinct()

  if compteur == 0:
    print("arret")
    boucle = False

t1=time.time()
#le round de python interfère l'opérateur round de pspark sql (qui s'applique à des colonnes)
print("c'est la fin; durée totale de :",t1-t0)

13277898
14353355
1593964
79434
8
0
arret
c'est la fin; durée totale de : 694.192519903183
